In [ ]:
for text in llm("AI is going to", stream=True):
    print(text, end="", flush=True)


# results on the cpu using ctransformers
### - 27.4 seconds
### output:
<p>
have an enormous impact on the future of the industry.
This year, AI was all over the show floor and there are a number of companies that are using AI in their products. Here's a quick look at some of them:
Fulcrum is an automated, artificial intelligence-powered software solution that helps manufacturers improve productivity and efficiency by automating processes, removing errors, and freeing up employees to focus on more complex tasks.
Kapil Khera, co-founder and CEO of Fulcrum, told Inman: "I don't know a single company that doesn't have AI in their product. I think it's the most important thing to come out of 2018."
The company is currently working on machine learning tools and workflow applications with machine intelligence technology so you can really begin building these powerful AI-powered solutions. The full AI-focused ecosystem, with automation in product design and sophisticated analytics, will be available to all customers by the end of 2018.
Kapil Khera, co-founder and CEO of Fulcrum
Surely
</p>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:

def get_response_tiny_llama(text):
    path='/teamspace/studios/this_studio/TinyLlama-1.1B-Chat-v1.0-GPTQ'
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    prompt = text
    system_message = "You are a story writing assistant"
    prompt_template=f'''<|system|>
    {system_message}</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    '''

    print("\n\n*** Generate:")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return(tokenizer.decode(output[0]))

In [ ]:
print(get_response_tiny_llama("what is qlora?"))

# result on gpu 2.8 seconds

# handling the embedding model

In [8]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


/Users/matansharon/python/opensource_rag/rag_venv3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/matansharon/python/opensource_rag/rag_venv3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[-2.02644442e-04  8.14801529e-02  3.13617773e-02  2.92062038e-03
   2.61564888e-02  2.90739350e-02  7.82618448e-02 -1.80426741e-03
   1.01344325e-01 -4.51711416e-02  5.84349334e-02 -1.53200952e-02
   5.49956597e-02 -9.86435115e-02 -3.50252502e-02  8.45677778e-03
   1.58608183e-02  1.05626723e-02 -3.42709646e-02 -4.75060102e-03
   9.99022275e-02 -2.06018873e-02 -4.47837524e-02  3.12135480e-02
  -1.19240806e-02 -5.15015796e-02 -1.33605348e-02  1.89621467e-02
   9.76809934e-02 -5.44110946e-02 -3.43313552e-02  8.12905654e-02
   4.88120094e-02 -1.10284314e-02  2.13518254e-02  1.27189681e-02
  -1.43967643e-02  3.62864546e-02 -7.61233866e-02  3.23293880e-02
   2.08103228e-02 -4.22016010e-02  9.12907198e-02  2.08530612e-02
  -3.08016818e-02 -8.38505626e-02  1.30891707e-02 -3.00631206e-02
   4.11228724e-02 -1.27495319e-01 -7.78026953e-02 -3.93412225e-02
   1.52589928e-03 -2.80108713e-02  3.41662727e-02  1.46712167e-02
  -7.71653578e-02  1.63619578e-01  4.11294997e-02 -5.24459928e-02
  -4.18772

In [23]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=[]
    for page in pages:
        res.append(page.extract_text())
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')


738
270


In [28]:
import chromadb
client=chromadb.Client()
import os
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=os.environ.get("HuggingFace_API_KEY"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection=client.get_or_create_collection('test',embedding_function=huggingface_ef)


In [33]:
#write a function that recive a file path of a jason and read it
def read_json_file(file_path):
    import json
    with open(file_path) as f:
            data = json.load(f)
    return data


AttributeError: 'Collection' object has no attribute 'insert_many'